In [1]:
import sys
!{sys.executable} -m pip install -U vllm -v --timeout 100


Using pip 25.0.1 from /opt/conda/lib/python3.11/site-packages/pip (python 3.11)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.9/474.9 MB 59.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.4/800.4 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 120.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.

In [2]:
import os
os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

MODEL_DIR = "/home/dataset-assist-0/data/models/Qwen2.5-14B-Instruct"  # 改成你真实的模型目录（有config.json）

from vllm import LLM
llm = LLM(
    model=MODEL_DIR,
    tensor_parallel_size=2,   # 你2张A800，建议=2；如果出错先改成1
    dtype="auto",
    max_model_len=32768,
)

print("llm loaded")


ModuleNotFoundError: No module named 'vllm'

In [2]:
!pip -q install -U pymupdf


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [16]:
import sys
!{sys.executable} -m pip -q install -U pandas


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip -q install -U transformers


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [26]:
from pathlib import Path

INPUT_DIR = Path("/home/dataset-assist-0/annual-reports")   # 放年报PDF的目录
OUTPUT_DIR = Path("/home/dataset-assist-0/annual-reports_output")  # 输出目录（改成你的）
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

pdf_paths = sorted(INPUT_DIR.glob("*.pdf"))
len(pdf_paths), [p.name for p in pdf_paths[:5]]


(8,
 ['英伟达-2018-Annual-Report.pdf',
  '英伟达-2019-Annual-Report.pdf',
  '英伟达-2020-Annual-Report.pdf',
  '英伟达-2021-Annual-Report.pdf',
  '英伟达-2022-Annual-Report.pdf'])

In [27]:
import fitz  # pymupdf

def pdf_to_text(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    parts = []
    for page in doc:
        t = page.get_text("text")
        if t and t.strip():
            parts.append(t)
    return "\n\n".join(parts)


In [28]:
from transformers import AutoTokenizer
from vllm import SamplingParams

# 你已加载过 llm 的话，这里只要保证 MODEL_DIR 和 tokenizer 有
MODEL_DIR = "/home/dataset-assist-0/data/models/Qwen2.5-14B-Instruct"  # 改成你的模型目录
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)

MAX_MODEL_LEN = 32768
RESERVE_FOR_OUTPUT = 1500

def chunk_by_tokens(text, chunk_tokens=6000, overlap_tokens=300):
    ids = tokenizer.encode(text)
    step = chunk_tokens - overlap_tokens
    chunks = []
    for start in range(0, len(ids), step):
        end = min(start + chunk_tokens, len(ids))
        chunks.append(tokenizer.decode(ids[start:end]))
        if end >= len(ids):
            break
    return chunks

def build_one_shot_prompt(annual_text: str, instruction: str) -> str:
    return f"""你是一名严谨的财报分析师，只能依据给定原文写报告。

【指令】
{instruction}

【原文】
{annual_text}
"""

def build_map_prompts(chunks):
    prompts = []
    for i, ch in enumerate(chunks, 1):
        prompts.append(f"""
你将得到一段年报原文。请严格只根据原文提取信息，不要编造。
输出必须是JSON（不要多余文字），字段如下：
- chunk_id: "c{i:03d}"
- facts: 3~8条事实要点（尽量包含数字）
- financial_metrics: 若出现财务指标，用数组输出，例如 {{"name":"营收","value":"...","period":"..."}}
- major_events: 0~5条重大事项
- risks: 0~5条风险点
- quotes: 2~5条原文短句引用（用于支撑事实/数字）

【原文】
{ch}
""".strip())
    return prompts

def build_reduce_prompt(cards, instruction: str) -> str:
    return f"""
你将得到若干段JSON卡片（来自年报原文的抽取结果）。
请严格按照【指令】写最终短report。

严格规则：
- 只能使用卡片中出现的事实/数字/事件；不得新增、不得猜测。
- 若信息缺失，请写“原文未披露”。
- 每条关键数字或结论后加来源chunk_id，如（来源：c003,c007）。

【指令】
{instruction}

【卡片】
{chr(10).join(cards)}
""".strip()


In [29]:
def generate_report_for_text(annual_text: str, llm, instruction: str,
                             chunk_tokens=6000, overlap_tokens=300):
    total_tokens = len(tokenizer.encode(annual_text))
    can_one_shot = total_tokens <= (MAX_MODEL_LEN - RESERVE_FOR_OUTPUT)

    if can_one_shot:
        prompt = build_one_shot_prompt(annual_text, instruction)
        out = llm.generate([prompt], SamplingParams(temperature=0.2, max_tokens=1100))[0].outputs[0].text
        return out, {"mode": "one_shot", "total_tokens": total_tokens, "chunks": 0}

    # Map-Reduce
    chunks = chunk_by_tokens(annual_text, chunk_tokens=chunk_tokens, overlap_tokens=overlap_tokens)
    map_prompts = build_map_prompts(chunks)

    map_sampling = SamplingParams(temperature=0.1, max_tokens=900)
    map_outputs = llm.generate(map_prompts, map_sampling)
    cards = [o.outputs[0].text for o in map_outputs]

    reduce_prompt = build_reduce_prompt(cards, instruction)
    reduce_sampling = SamplingParams(temperature=0.2, max_tokens=1200)
    report = llm.generate([reduce_prompt], reduce_sampling)[0].outputs[0].text

    return report, {"mode": "map_reduce", "total_tokens": total_tokens, "chunks": len(chunks)}


In [32]:
from tqdm.auto import tqdm
import json

instruction = (
    "请把企业年报写成【800字以内】的短report，必须包含小标题："
    "1 公司概况；2 经营亮点；3 核心财务（尽量列数字）；4 风险提示；5 一句话结论。"
    "要求：只根据原文，不要编造数字/事件/年份；若缺失请写“原文未披露”。"
)

results = []

for pdf_path in tqdm(pdf_paths):
    out_txt = OUTPUT_DIR / (pdf_path.stem + "_short_report.txt")
    out_meta = OUTPUT_DIR / (pdf_path.stem + "_meta.json")

    # 已有结果就跳过（可删掉这段让它覆盖重跑）
    if out_txt.exists() and out_meta.exists():
        continue

    annual_text = pdf_to_text(str(pdf_path))
    report, meta = generate_report_for_text(annual_text, llm, instruction)

    out_txt.write_text(report, encoding="utf-8")
    out_meta.write_text(json.dumps({"file": pdf_path.name, **meta}, ensure_ascii=False, indent=2), encoding="utf-8")
    results.append({"file": pdf_path.name, **meta})

results[:3], len(results)


  0%|          | 0/8 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (165964 > 131072). Running this sequence through the model will result in indexing errors

Adding requests: 100%|██████████| 30/30 [00:00<00:00, 91.11it/s]

Processed prompts: 100%|██████████| 30/30 [00:40<00:00,  1.36s/it, est. speed input: 4371.75 toks/s, output: 641.73 toks/s]

Adding requests: 100%|██████████| 1/1 [00:00<00:00, 13.07it/s]

Adding requests: 100%|██████████| 22/22 [00:00<00:00, 68.65it/s]

Processed prompts: 100%|██████████| 22/22 [00:33<00:00,  1.52s/it, est. speed input: 3951.43 toks/s, output: 585.30 toks/s]

Adding requests: 100%|██████████| 1/1 [00:00<00:00, 15.90it/s]

Adding requests: 100%|██████████| 29/29 [00:00<00:00, 89.84it/s]

Processed prompts: 100%|██████████| 29/29 [00:40<00:00,  1.39s/it, est. speed input: 4318.44 toks/s, output: 641.19 toks/s]

Adding requests: 100%|██████████| 1/1 [00:00<00:00, 12.94it/s]

Adding re

([{'file': '英伟达-2018-Annual-Report.pdf',
   'mode': 'map_reduce',
   'total_tokens': 165964,
   'chunks': 30},
  {'file': '英伟达-2019-Annual-Report.pdf',
   'mode': 'map_reduce',
   'total_tokens': 123224,
   'chunks': 22},
  {'file': '英伟达-2020-Annual-Report.pdf',
   'mode': 'map_reduce',
   'total_tokens': 161933,
   'chunks': 29}],
 8)

In [33]:
import pandas as pd
df = pd.DataFrame(results)
df


,file,mode,total_tokens,chunks
0,英伟达-2018-Annual-Report.pdf,map_reduce,165964,30
1,英伟达-2019-Annual-Report.pdf,map_reduce,123224,22
2,英伟达-2020-Annual-Report.pdf,map_reduce,161933,29
3,英伟达-2021-Annual-Report.pdf,map_reduce,132114,24
4,英伟达-2022-Annual-Report.pdf,map_reduce,164904,29
5,英伟达-2023-Annual-Report.pdf,map_reduce,143118,26
6,英伟达-2024-Annual-Report.pdf,map_reduce,152066,27
7,英伟达-2025-Annual-Report.pdf,map_reduce,160719,29
